<a href="https://colab.research.google.com/github/chefinha-tech/acad_control/blob/main/verificador_certificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ssl
import socket
import datetime

def verificar_certificado_site(hostname, port=443):
    """
    Verifica a validade do certificado SSL/TLS de um site.

    Args:
        hostname (str): O nome do host (domínio) do site.
        port (int): A porta (padrão 443 para HTTPS).

    Returns:
        dict: Um dicionário com o status do certificado e informações.
    """
    context = ssl.create_default_context()
    try:
        with socket.create_connection((hostname, port)) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()

                # Extrair datas de validade
                not_before_str = cert['notBefore']
                not_after_str = cert['notAfter']

                # Converter para objetos datetime
                # Exemplo de formato: 'Jun 24 10:00:00 2025 GMT'
                # Pode variar, então é bom ter um tratamento de erro ou regex para robustez
                try:
                    not_before = datetime.datetime.strptime(not_before_str, '%b %d %H:%M:%S %Y %Z')
                    not_after = datetime.datetime.strptime(not_after_str, '%b %d %H:%M:%S %Y %Z')
                except ValueError:
                    # Tratar outros formatos ou usar uma biblioteca mais robusta como 'parsedatetime'
                    return {
                        "status": "Erro na data",
                        "hostname": hostname,
                        "detalhes": "Não foi possível parsear as datas de validade do certificado."
                    }

                current_time = datetime.datetime.now(datetime.timezone.utc) # Comparar com UTC

                status = "Válido"
                if current_time < not_before:
                    status = "Ainda não válido (data futura)"
                elif current_time > not_after:
                    status = "Expirado"

                return {
                    "status": status,
                    "hostname": hostname,
                    "not_before": not_before.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "not_after": not_after.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "dias_restantes": (not_after - current_time).days if status == "Válido" else None,
                    "issuer": dict(x[0] for x in cert['issuer']) # Informações do emissor
                }

    except ssl.SSLCertVerificationError as e:
        return {
            "status": "Erro de verificação SSL",
            "hostname": hostname,
            "detalhes": str(e)
        }
    except socket.gaierror:
        return {
            "status": "Erro de DNS",
            "hostname": hostname,
            "detalhes": "Não foi possível resolver o nome do host."
        }
    except ConnectionRefusedError:
        return {
            "status": "Conexão recusada",
            "hostname": hostname,
            "detalhes": "A conexão foi recusada pelo servidor. Porta incorreta ou firewall?"
        }
    except Exception as e:
        return {
            "status": "Erro inesperado",
            "hostname": hostname,
            "detalhes": str(e)
        }

# --- Como usar a automação ---
if __name__ == "__main__":
    sites_para_verificar = [
        "google.com",
        "certifica.com.br",
        "globalsign.com",
        "digicert.com",
        "expired.badssl.com", # Exemplo de site com certificado expirado
        "google.com:80", # Porta incorreta para HTTPS
        "sitequenaoexiste12345.com" # Exemplo de site que não existe
    ]

    print("Iniciando verificação de certificados SSL/TLS...\n")
    for site in sites_para_verificar:
        print(f"Verificando: {site}")
        resultado = verificar_certificado_site(site)
        for key, value in resultado.items():
            print(f"  {key}: {value}")
        print("-" * 30)

    # Exemplo de verificação de revogação (mais complexo e fora do escopo deste exemplo simples)
    # Para verificação de revogação (CRL/OCSP), seria necessário implementar a lógica
    # para consultar os endpoints de OCSP ou baixar e parsear as CRLs da AC.
    # Isso geralmente exige bibliotecas mais avançadas como `pyOpenSSL` ou `cryptography`
    # e uma compreensão mais profunda da PKI.